In [1]:
import sys
import os
import logging
sys.path.append(r"..")
from utils import *
from model import *
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score,roc_auc_score,f1_score,recall_score
from scipy import sparse
from scipy.sparse import csr_matrix, coo_matrix

from collections import defaultdict
from collections import Counter

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.client import device_lib
#print(tf.__version__)
#print(tf.test.is_built_with_gpu_support)
#print(tf.test.is_gpu_available())
#print(device_lib.list_local_devices())
os.environ["CUDA_VISIBLE_DEVICES"] = '0' #use GPU with ID=0
gpus = tf.config.experimental.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(gpus[0], True)
#对需要进行限制的GPU进行设置
# tf.config.experimental.set_virtual_device_configuration(gpus[0],
#                                                       [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)])
# gpus

In [2]:
user_ids = pd.read_pickle(f"{pickle_path}/user_ids_relencode.pickle")
user = pd.read_csv(train_preliminary_p + "user.csv", encoding='utf-8')
user_ids = user_ids.merge(user, how='left', on='user_id')
save_pickle(user_ids, f"{pickle_path}/train_user_ids.pickle")
#user_ids = pd.read_pickle(f"{pickle_path}/train_user_ids.pickle") #保存从8min缩小到3min
print(user_ids.shape)
user_ids.head(2)

(1900000, 32)


,user_id,creative_id,ad_id,product_id,product_category,advertiser_id,industry,click_times,time,creative_id_times,...,max_clicktimes,min_clicktimes,mean_time,max_time,min_time,time_len,time_clicktimes,time_creativeids,age,gender
0,1,"[642275, 170988, 685687, 1307085, 100774, 5953...","[581171, 156554, 620126, 1166887, 92441, 55685...","[1, 125, 1, 1, 1132, 1, 1, 1, 1, 1244, 55, 55,...","[5, 2, 5, 5, 2, 18, 5, 5, 18, 2, 2, 2, 2]","[6411, 8522, 26781, 12996, 10113, 13007, 15341...","[323, 7, 104, 323, 1, 323, 73, 214, 64, 235, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2]","[20, 20, 20, 39, 40, 43, 46, 52, 60, 64, 64, 7...","[3872964, 1123872, 3935014, 706993, 226871, 37...",...,2,1,47.461538,76,20,10,"[3, 1, 1, 1, 1, 1, 1, 2, 1, 2]","[3, 1, 1, 1, 1, 1, 1, 2, 1, 1]",4.0,1.0
1,2,"[52634, 128035, 32734, 478744, 10372, 349730, ...","[49388, 117016, 31581, 435462, 11432, 318236, ...","[76, 69, 118, 118, 1193, 76, 1, 1064, 1, 1, 10...","[2, 2, 2, 2, 2, 2, 18, 2, 18, 18, 2, 18, 2, 18...","[20681, 9438, 16619, 23545, 705, 20681, 31504,...","[315, 235, 7, 7, 314, 315, 47, 7, 47, 47, 239,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[10, 11, 14, 17, 28, 28, 28, 38, 38, 39, 41, 4...","[3670932, 575824, 3013599, 3644043, 316325, 34...",...,2,1,53.822222,90,10,28,"[1, 1, 1, 1, 3, 2, 1, 1, 3, 2, 1, 4, 1, 4, 1, ...","[1, 1, 1, 1, 3, 2, 1, 1, 3, 2, 1, 4, 1, 4, 1, ...",10.0,1.0


In [3]:
creative_id_em = load_pickle(f"{path_save}/creative_id_w2v_matrix2.pkl")
ad_id_em = load_pickle(f"{path_save}/ad_id_w2v_matrix2.pkl")
advertiser_id_em = load_pickle(f"{path_save}/advertiser_id_w2v_matrix2.pkl")
product_id_em = load_pickle(f"{path_save}/product_id_w2v_matrix2.pkl")
industry_em = load_pickle(f"{path_save}/industry_w2v_matrix2.pkl")
product_category_em = load_pickle(f"{path_save}/product_category_w2v_matrix2.pkl")
# click_times_em = load_pickle(f"{path_save}/click_times_w2v_matrix2.pkl")
# time_em = load_pickle(f"{path_save}/time_w2v_matrix2.pkl")

creative_id_times_em = load_pickle(f"{path_save}/creative_id_times_w2v_matrix2.pkl")
ad_id_times_em = load_pickle(f"{path_save}/ad_id_times_w2v_matrix2.pkl")
product_id_times_em = load_pickle(f"{path_save}/product_id_times_w2v_matrix2.pkl")
advertiser_id_times_em = load_pickle(f"{path_save}/advertiser_id_times_w2v_matrix2.pkl")
product_category_times_em = load_pickle(f"{path_save}/product_category_times_w2v_matrix2.pkl")
industry_times_em = load_pickle(f"{path_save}/industry_times_w2v_matrix2.pkl")

# creative_id_t_em = load_pickle(f"{path_save}/creative_id_t_w2v_matrix2.pkl")
# # ad_id_t_em = load_pickle(f"{path_save}/ad_id_t_w2v_matrix2.pkl")
# product_id_t_em = load_pickle(f"{path_save}/product_id_t_w2v_matrix2.pkl")
# advertiser_id_t_em = load_pickle(f"{path_save}/advertiser_id_t_w2v_matrix2.pkl")
# product_category_t_em = load_pickle(f"{path_save}/product_category_t_w2v_matrix2.pkl")
# industry_t_em = load_pickle(f"{path_save}/industry_t_w2v_matrix2.pkl")

time_clicktimes_em = load_pickle(f"{path_save}/time_clicktimes_w2v_matrix2.pkl")
time_creativeids_em = load_pickle(f"{path_save}/time_creativeids_w2v_matrix2.pkl")

w2v_features = [
    {'name':'creative_id', 'size':128, 'windows':5, 'min_count':1, 'version':2, 'max_len':128, 'em':creative_id_em},
    {'name':'ad_id', 'size':128, 'windows':5, 'min_count':1, 'version':2, 'max_len':128, 'em':ad_id_em},
    {'name':'advertiser_id', 'size':64, 'windows':5, 'min_count':1, 'version':2, 'max_len':128, 'em':advertiser_id_em},
    {'name':'product_id', 'size':64, 'windows':5, 'min_count':1, 'version':2, 'max_len':128, 'em':product_id_em},
    {'name':'industry', 'size':32, 'windows':5, 'min_count':1, 'version':2, 'max_len':128, 'em':industry_em},
    {'name':'product_category', 'size':16, 'windows':5, 'min_count':1, 'version':2, 'max_len':128, 'em':product_category_em},
#     {'name':'time', 'size':16, 'windows':5, 'min_count':1, 'version':2, 'max_len':128, 'em':time_em},
#     {'name':'click_times', 'size':8, 'windows':5, 'min_count':1, 'version':2, 'max_len':128, 'em':click_times_em},
    
    {'name':'creative_id_times', 'size':128, 'windows':5, 'min_count':1, 'version':2, 'max_len':128, 'em':creative_id_times_em},
    #{'name': 'ad_id_times', 'size': 128, 'windows': 5, 'min_count': 1, 'version': 2, 'max_len':128, 'em':ad_id_times_em},
    {'name':'product_id_times', 'size':64, 'windows':5, 'min_count':1, 'version':2, 'max_len':128, 'em':product_id_times_em},
    {'name':'advertiser_id_times', 'size':64, 'windows':5, 'min_count':1, 'version':2, 'max_len':128, 'em':advertiser_id_times_em},
    {'name':'product_category_times', 'size':32, 'windows':5, 'min_count':1, 'version':2,'max_len':128, 'em':product_category_times_em},
    {'name':'industry_times', 'size':32, 'windows':5, 'min_count':1, 'version':2,'max_len':128, 'em':industry_times_em},   
    
    
    #{'name':'time_clicktimes', 'size':32, 'windows':5, 'min_count':1, 'version':2, 'vocab_size':5000,'max_len':128,'em':time_clicktimes_em},
    #{'name':'time_creativeids', 'size':32, 'windows':5, 'min_count':1, 'version':2,'vocab_size':5000, 'max_len':128,'em': time_creativeids_em},
    
]
#dense_features = ['creative_id_len', 'ad_id_len', 'product_id_len', 'product_category_len', 'advertiser_len', 'industry_len','time_len',
#                  'mean_clicktimes', 'max_clicktimes', 'min_clicktimes', 'mean_time', 'max_time', 'min_time']  
dense_features = []

#print(creative_id_em.shape,ad_id_em.shape,advertiser_id_em.shape,product_id_em.shape)
#print(industry_em.shape,product_category_em.shape,click_times_em.shape,time_em.shape)

In [4]:
def lstm_model(units,num_classes,w2v_features, dense_features):
    inputs_dict = dict()
    embed_layer_list = []
    for w2v_f in w2v_features:
        em_name = w2v_f['name']
        em_size = w2v_f['em'].shape[0]
        em_dim = w2v_f['size']
        em_m = w2v_f['em']
        max_len = w2v_f['max_len']

        inputs = keras.Input(shape=(max_len,), name=em_name)
        inputs_dict[em_name] = inputs

        embed_layer_list.append( keras.layers.Embedding(
                em_size, em_dim, input_length=max_len, trainable=False, weights=[em_m],mask_zero=True)(inputs))
    embed_output = keras.layers.concatenate(embed_layer_list, axis=-1)
    lstm_output = keras.layers.Bidirectional(keras.layers.LSTM(units,return_sequences=True))(embed_output)
    
    #lstm_output = layers.GlobalMaxPooling1D()(lstm_output)
                                     
    
    lstm_output = layers.concatenate([layers.GlobalAveragePooling1D()(lstm_output),
                                      layers.GlobalMaxPooling1D()(lstm_output),
                                     ], axis=-1)
    lstm_output = layers.Dropout(0.2)(lstm_output)
    fc = keras.layers.Dense(units, activation='relu')(lstm_output)
    #数值型特征
    numeric_list = []
    for den_f in dense_features:
        inputs = keras.Input(shape=(1,), name=den_f)
        inputs_dict[den_f] = inputs
        numeric_list.append(inputs)   
    if dense_features != []:
        numeric_output = keras.layers.concatenate(numeric_list, axis=-1)
        lstm_numeric_output = keras.layers.concatenate([fc,numeric_output], axis=-1)
    else:
        lstm_numeric_output = fc
    
    
    outputs = keras.layers.Dense(num_classes, activation='softmax')(lstm_numeric_output)
    
    
    model = keras.Model(inputs=inputs_dict, outputs=outputs)
    model.compile(optimizer = keras.optimizers.Adam(0.001),
              loss = keras.losses.sparse_categorical_crossentropy,
              metrics = ['accuracy'])
    return model

In [5]:
def input_fn(feature_dict, label=None, epochs=5, shuffle=True, batch_size=64, fit_key='train'):
    if fit_key == 'train':
        dataset = tf.data.Dataset.from_tensor_slices((feature_dict, label))
    else:
        dataset = tf.data.Dataset.from_tensor_slices((feature_dict))
    if shuffle:
        dataset = dataset.shuffle(100*batch_size)
    dataset = dataset.repeat(epochs).batch(batch_size)
    return dataset

In [6]:
train_split = [0,810000]
val_split = [810000, 900000]
test_split = [900000]
train_feature_dict = dict()
val_feature_dict = dict()
test_feature_dict = dict()
for fea in w2v_features:
    name = fea['name']
    max_length = fea['max_len']
    print(name)
    test_feature_dict[name] = keras.preprocessing.sequence.pad_sequences(
        user_ids[name][test_split[0]:],value = 0,padding = 'post',maxlen = max_length )
for fea in dense_features:
    print(fea)
    test_feature_dict[fea] = user_ids[fea][test_split[0]:]


kfolder = KFold(n_splits=n_fold, shuffle=True, random_state=seed)
kfold = kfolder.split(train_x, train_y)
num_classes = 2
units = 128
for train_index, vali_index in kfold:
    for fea in w2v_features:
        name = fea['name']
        max_length = fea['max_len']
        print(name)
        train_feature_dict[name] = keras.preprocessing.sequence.pad_sequences(
            user_ids[name][train_index],value = 0,padding = 'post',maxlen = max_length )
        val_feature_dict[name] = keras.preprocessing.sequence.pad_sequences(
            user_ids[name][vali_index],value = 0,padding = 'post',maxlen = max_length )
    for fea in dense_features:
        print(fea)
        train_feature_dict[fea] = user_ids[fea][train_index]
        val_feature_dict[fea] = user_ids[fea][vali_index]
    gender_train_label = np.array(user_ids['gender'][train_index])
    gender_val_label = np.array(user_ids['gender'][vali_index])

    age_train_label = np.array(user_ids['age'][train_index])
    age_val_label = np.array(user_ids['age'][vali_index])
    
    
    

    gender_model = lstm_model(units, num_classes, w2v_features, [])
    train_dataset = input_fn(train_feature_dict, gender_train_label-1, epochs=5, shuffle=True, batch_size=128)
    val_dataset = input_fn(val_feature_dict, gender_val_label-1, epochs=1, shuffle=False, batch_size=1024)
    gender_model.fit(train_dataset, validation_data=val_dataset)

    
    test_dataset = input_fn(test_feature_dict, epochs=1, shuffle=False, batch_size=1024, fit_key='predict')
    gender_prob = gender_model.predict(test_dataset)
    gender_val_prob = gender_model.predict(val_dataset)
    print(gender_prob.shape,gender_val_prob.shape)
    tune_weight = search_weight(gender_val_label-1, gender_val_prob, init_weight=[1.0]*2,class_num=2, step=0.001)
    
    

creative_id
ad_id
advertiser_id
product_id
industry
product_category
creative_id_times
product_id_times
advertiser_id_times
product_category_times
industry_times
time_clicktimes


In [ ]:
#def age_model_run():
num_classes = 10
units = 128
age_model = lstm_model(units, num_classes, w2v_features, [])
#age_model.summary()
train_dataset = input_fn(train_feature_dict, age_train_label-1, epochs=6, shuffle=True, batch_size=128)
val_dataset = input_fn(val_feature_dict, age_val_label-1, epochs=1, shuffle=False, batch_size=1024)
age_model.fit(train_dataset, validation_data=val_dataset)

test_dataset = input_fn(test_feature_dict, epochs=1, shuffle=False, batch_size=1024, fit_key='predict')
age_prob = age_model.predict(test_dataset)
age_val_prob = age_model.predict(val_dataset)
print(age_prob.shape,age_val_prob.shape)
age_tune_weight = search_weight(age_val_label-1, age_val_prob, init_weight=[1.0]*10,class_num=10, step=0.001)
print(age_tune_weight)


In [ ]:
# from multiprocessing import Pool
# p = Pool(1)
# p.apply_async(age_model_run)
# p.close()
# p.join()

In [ ]:
gender_prob_tune = np.array(tune_weight)*gender_prob
gender_pre = np.argmax(gender_prob_tune,axis=1) + 1

age_prob_tune = np.array(age_tune_weight)*age_prob
age_pre = np.argmax(age_prob_tune,axis=1) + 1

np.save(f"{sub_path}/age_prob.npy", age_prob)
np.save(f"{sub_path}/gender_prob.npy", gender_prob)


sub = pd.DataFrame()
sub['user_id'] = range(3000001,4000001)
sub['predicted_age'] = age_pre
sub['predicted_gender'] = gender_pre
print('ok！')
sub.to_csv(f"{sub_path}/submission.csv", index=False, encoding='utf-8')

In [ ]:
# # 放入cpu中
# with tf.device("/CPU:0"):
#     creative_id_em = tf.constant(creative_id_em)
#     ad_id_em = tf.constant(ad_id_em)

np.save(f"{sub_path}/val_age_prob.npy", age_val_prob)
np.save(f"{sub_path}/val_gender_prob.npy", gender_val_prob)

In [ ]:
# tf.keras.backend.clear_session()
# import random
# #del gender_model
# del age_model
# def reset_seeds():
#     np.random.seed(1)
#     random.seed(2)
#     if tf.__version__[0] == '2':
#         tf.random.set_seed(3)
#     else:
#         tf.set_random_seed(3)
#     print("RANDOM SEEDS RESET")
    
# tf.compat.v1.reset_default_graph()
# reset_seeds()    

In [ ]:
gender_model.summary()